In [ ]:
%%configure -f
{
    "defaultLakehouse": {
        "name": "Config"
    }
}

In [ ]:
%run ./nb_config_env

In [ ]:
import requests
import json
# import time
from time import sleep 
from pyspark.sql.functions import *

In [ ]:
def fn_get_refresh(url, payload, header):
    resp = requests.get(
        url = url,
        json = payload,
        headers = header
    )

    data = json.loads(resp.text)
    st = data['value'][0]['status']

    #print(resp.text)
    #print(resp.content) 
    #print(resp.json())  # Convert response JSON to Python object

    if resp.status_code == 200:
        rv = json.loads(resp.text)['value']
    else:
        rv = f"Error: {resp.status_code} - {resp.text}"

    return st

In [ ]:
def fn_post_refresh(url, payload, header):
    resp = requests.post(
        url = url,
        json = payload,
        headers = header
    )

    resp.raise_for_status()
    print (resp)

    if resp.status_code in (201,202):
        print("Post request successful.")
    else:
        print(f"Error: {resp.status_code} - {resp.text}")
        
    return

In [ ]:
source_file = "synapse_params.json"
source_path = path_lh_config + '/Files/Config/'
source_url = source_path + source_file
df = spark.read.json(source_url)

wsid = df.select("PBIAppWorkspaceCD").head()[0]
dsid = df.select("PBIDatasetCD").head()[0]

In [ ]:
# Get Bearer Token, url and header
token_string = mssparkutils.credentials.getToken("https://api.fabric.microsoft.com/")

urlpost = 'https://api.powerbi.com/v1.0/myorg/groups/' + wsid + '/datasets/' + dsid + '/refreshes'
urlget  = 'https://api.powerbi.com/v1.0/myorg/groups/' + wsid + '/datasets/' + dsid + '/refreshes?$top=1'

payloadpost = {"Hello": "World"}
payloadget = {"Hello": "World"}
header = {'Content-Type':'application/json','Authorization': f'Bearer {token_string}'}
RefreshStatus = 'Unknown'
LoopWaitTimeSec = 10
LoopCount = 1

In [ ]:
fn_post_refresh(urlpost, payloadpost, header)

In [ ]:
RefreshStatus = fn_get_refresh(urlget, payloadget, header)
# print(RefreshStatus)
while LoopCount <= 20:
    print(LoopCount)
    LoopCount += 1
    sleep(LoopWaitTimeSec)
    RefreshStatus = fn_get_refresh(urlget, payloadget, header)
    print(RefreshStatus)
    if RefreshStatus != 'Unknown':
        break

In [ ]:
if RefreshStatus != 'Completed':
    print('Refresh process is completed with status:' + RefreshStatus)
    1/0